##### Импортируем необходимые библиотеки

In [119]:
import torch
import transformers as ppb
import requests
import pandas as pd
import numpy as np
import zipfile
import io
from requests.auth import HTTPBasicAuth
from gensim.models import Word2Vec
import gensim.downloader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

##### Делаем запрос на Kaggle

In [2]:
username = 'dariapetunina'
key = '76e3b0577a670c4404d4f509635a6a3f'

In [3]:
url = 'https://www.kaggle.com/api/v1/datasets/download/razanaqvi14/real-and-fake-news'

In [4]:
response = requests.get(url, auth=HTTPBasicAuth(username, key))
response

<Response [200]>

##### Работа с архивом

In [5]:
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    print(z.namelist())

['Fake.csv', 'True.csv']


In [6]:
csv_fake = z.namelist()[0]
csv_true = z.namelist()[1]

In [7]:
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open(csv_fake) as f:
        df_fake = pd.read_csv(f, header=0)

In [8]:
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open(csv_true) as f:
        df_true = pd.read_csv(f, header=0)

##### Первичная работа с датафреймами

In [9]:
df_fake.head(3)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"


In [10]:
df_fake.shape

(23481, 4)

In [11]:
df_true.head(3)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"


In [12]:
df_true.shape

(21417, 4)

##### Кодирование датафреймов

In [13]:
df_fake['flag'] = 0
df_fake.head(3)

,title,text,subject,date,flag
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0


In [14]:
df_true['flag'] = 1
df_true.head(3)

,title,text,subject,date,flag
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1


##### Объединение датафреймов

In [15]:
df = pd.concat([df_true, df_fake], axis=0, ignore_index=True)
df.head(3)

,title,text,subject,date,flag
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1


In [16]:
df.shape

(44898, 5)

##### Первичная обработка данных

In [17]:
# Проверка типа данных и пропущенных значений
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   flag     44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


title      0
text       0
subject    0
date       0
flag       0
dtype: int64

In [18]:
df['text'] = df['text'].astype(str)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   flag     44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [20]:
# Проверка дубликатов
num_duplicates = df.duplicated().sum()
print(f"Количество дубликатов: {num_duplicates}")
df.shape

Количество дубликатов: 209


(44898, 5)

In [21]:
df_unique = df.drop_duplicates()
df_unique.shape

(44689, 5)

##### Разделение на обучающую и тестовую выборки

In [22]:
df_unique.head(3)

,title,text,subject,date,flag
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1


In [23]:
# Баланс классов
df_unique['flag'].value_counts()

0    23478
1    21211
Name: flag, dtype: int64

In [24]:
X = df_unique['text']
y = df_unique['flag']

In [25]:
print(X)

0        WASHINGTON (Reuters) - The head of a conservat...
1        WASHINGTON (Reuters) - Transgender people will...
2        WASHINGTON (Reuters) - The special counsel inv...
3        WASHINGTON (Reuters) - Trump campaign adviser ...
4        SEATTLE/WASHINGTON (Reuters) - President Donal...
                               ...                        
44893    21st Century Wire says As 21WIRE reported earl...
44894    21st Century Wire says It s a familiar theme. ...
44895    Patrick Henningsen  21st Century WireRemember ...
44896    21st Century Wire says Al Jazeera America will...
44897    21st Century Wire says As 21WIRE predicted in ...
Name: text, Length: 44689, dtype: object


In [26]:
print(y)

0        1
1        1
2        1
3        1
4        1
        ..
44893    0
44894    0
44895    0
44896    0
44897    0
Name: flag, Length: 44689, dtype: int64


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(35751,) (8938,) (35751,) (8938,)


### Обучение моделей

* **Обучаем векторизатор текста на тренировочных данных**
* **Строим прогноз и оцениваем качество модели на тренировочных и тестовых данных**

#### TF-IDF

**TF-IDF** (от англ. Term Frequency-Inverse Document Frequency) — это статистическая мера, которая оценивает важность слова в конкретном документе относительно всего корпуса документов. Вес слова растёт, если оно часто встречается в данном документе (TF — частота термина), и уменьшается, если оно часто встречается во многих документах корпуса (IDF — обратная частота документа)

In [29]:
# Обучаем модель на обучающей выборке
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(X_train)

TfidfVectorizer(stop_words='english')

In [30]:
# Преобразуем текстовые значения из обучающей и тестовой выборок в числовые векторы
Xtrain = tfidf.transform(X_train)
Xtest = tfidf.transform(X_test)

In [31]:
Xtrain.shape, Xtest.shape

((35751, 111249), (8938, 111249))

* Очень много фичей: **есть вероятность переобучения**

##### Обучение логистической регрессии

In [32]:
# Обучаем на обучающей выборке
lr_clf = LogisticRegression()
lr_clf.fit(Xtrain, y_train)

LogisticRegression()

In [33]:
# Строим предсказания с помощью фичей на обучающей и тестовой выборках
lr_train_pred = lr_clf.predict(Xtrain)
lr_test_pred = lr_clf.predict(Xtest)

##### Оценка качества модели

**Accuracy (точность)** — это метрика, которая показывает долю правильных предсказаний модели относительно общего числа предсказаний. Она рассчитывается как отношение суммы верных положительных (True Positive, TP) и верных отрицательных (True Negative, TN) предсказаний к общему числу предсказаний

In [34]:
accuracy_score(y_train, lr_train_pred), accuracy_score(y_test, lr_test_pred)

(0.9918603675421667, 0.9841127769075856)

##### Подбор оптимальных параметров для токенайзера

In [ ]:
# Создаем пайплайн
#pipeline = Pipeline([
    #('tfidf', TfidfVectorizer()),
    #('lr_clf', LogisticRegression(max_iter=1000))
#])

# Задаем сетку параметром для перебора
#param_grid = {
    #'tfidf__min_df': [1, 2, 5],
    #'tfidf__max_df': [0.7, 0.85, 1.0],
    #'tfidf__ngram_range': [(1,1), (1,2), (1,3)] 
#}

# Запускаем процесс кросс-валидации
#grid_search = GridSearchCV(pipeline, param_grid, cv = 5, scoring='neg_mean_squared_error', n_jobs=-1)

# Обучение
#grid_search.fit(X_train, y_train)

# Вытаскиваем наилучшие параметры
#print("Лучшие параметры:", grid_search.best_params_)

### Word2Vec

**Word2vec** — это набор моделей на основе искусственных нейронных сетей, предназначенных для преобразования слов из текста в векторные представления, которые отражают их семантические и синтаксические отношения

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [95]:
# Разбиваем строку на слова по пробелам
sent = [row.split() for row in X_train[0]]

# Размерность векторного представления (количество чисел в векторе)
HIDDEN = 100

model = Word2Vec(min_count = 20, # игнорируем слова, которые встречаются менее 20 раз
                     window = 2, # смотрим на 2 слова справа и 2 слова слева 
                     vector_size = HIDDEN, # размерность вектора 
                     sample = 6e-5, # уменьшаем влияние очень частых слов
                     alpha = 0.03, # начальная скорость обучения 
                     min_alpha = 0.0007, # мнинимальная скорость обучения 
                     negative = 20, # количество негативных примеров
                     workers = 2) # количество потоков для параллельного обучения

# Создаем словарь
model.build_vocab(sent, progress_per=10000)

# Обучение модели
model.train(sent, total_examples = model.corpus_count, epochs = 30, report_delay = 1)

(4234, 117210)

In [96]:
X_train.shape

(35751,)

* (количество **уникальных** слов, **общее** количество слов)

##### Рассчитаем средний вектор текста

Текст может состоять из многих слов, каждое из которых имеет свой вектор. **Усреднение этих векторов даёт один фиксированной размерности вектор, который отражает общий смысл текста**

In [97]:
def get_mean_w2v_vector(sentence):
    # Если вход — строка, разбиваем на слова
    if isinstance(sentence, str):
        words = sentence.split()
    # Если вход — список или массив слов, используем как есть
    elif isinstance(sentence, (list, tuple, np.ndarray)):
        words = sentence
    else:
        words = []

    Sum = 0
    Count = 0

    for w in words:
        if w in model.wv:
            Sum += model.wv[w]
            Count += 1

    if Count == 0:
        return np.zeros(model.vector_size)  # Возвращаем нулевой вектор нужной размерности

    return Sum / Count

In [98]:
# 1. Получаем Series векторов
train_vectors = X_train.map(get_mean_w2v_vector)
test_vectors = X_test.map(get_mean_w2v_vector)

# 2. Преобразуем Series в DataFrame
train_vec_df = pd.DataFrame(train_vectors.tolist(), columns=NewCols, index=X_train.index)
test_vec_df = pd.DataFrame(test_vectors.tolist(), columns=NewCols, index=X_test.index)

# 3. Объединяем с исходными данными
X_train = pd.concat([X_train, train_vec_df], axis=1)
X_test = pd.concat([X_test, test_vec_df], axis=1)

In [99]:
print(X_train.shape, X_test.shape)

(35751, 101) (8938, 101)


In [100]:
X_train.drop(['text'], axis=1, inplace=True)
X_test.drop(['text'], axis=1, inplace=True)

In [101]:
X_train.head(3)

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99
7041,-0.00862,0.003666,0.005190,0.005742,0.007467,-0.006168,0.001106,0.006047,-0.002840,-0.006174,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
5254,-0.00862,0.003666,0.005190,0.005742,0.007467,-0.006168,0.001106,0.006047,-0.002840,-0.006174,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
40178,-0.00240,0.001691,0.004084,0.005318,0.003333,-0.001708,0.000890,0.002975,-0.004402,-0.003310,...,0.000703,-0.000987,0.001047,-0.003366,-0.000206,0.002325,0.002696,-0.002042,-0.002561,0.000464


In [102]:
X_train.shape

(35751, 100)

##### Обучаем модель логистической регрессии

In [103]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)

lr_train_pred = lr_clf.predict(X_train)
lr_test_pred = lr_clf.predict(X_test)

accuracy_score(y_train, lr_train_pred), accuracy_score(y_test, lr_test_pred)

(0.7528740454812453, 0.7517341687178339)

### Предобученные модели из gensim

**Загружаем векторы слов из предобученных моделей**

In [106]:
# Список всех доступных предпобученных моделей
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [107]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

##### Пример вектора слов

In [110]:
glove_vectors['bank']

array([-0.026332,  1.2378  , -0.19281 , -1.1141  ,  0.10783 , -0.056344,
       -0.59693 , -0.14492 ,  1.1163  ,  1.1818  ,  0.8562  ,  0.65737 ,
       -3.4613  ,  0.10235 ,  0.26078 , -0.42548 ,  0.42242 ,  0.22929 ,
       -0.55839 ,  0.16926 , -0.78357 , -0.78385 , -0.64772 , -0.62314 ,
       -0.44248 ], dtype=float32)

### Предобученный BERT

In [111]:
!pip install transformers


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### DistilBERT

In [123]:
# For DistilBERT
# Достаем модель, токенацзер и предобученные веса
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [129]:
# Загружаем предобученную модель и токенайзер
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [132]:
pretrained_weights

'distilbert-base-uncased'

##### Токенизация

In [116]:
df_unique['text'].head(3)

0    WASHINGTON (Reuters) - The head of a conservat...
1    WASHINGTON (Reuters) - Transgender people will...
2    WASHINGTON (Reuters) - The special counsel inv...
Name: text, dtype: object

In [135]:
def truncate_to_512_bytes(text, max_bytes=512, encoding='utf-8'):
    if not isinstance(text, str):
        return text  # если не строка, возвращаем как есть
    encoded = text.encode(encoding)
    if len(encoded) <= max_bytes:
        return text
    truncated_encoded = encoded[:max_bytes]
    # Декодируем обратно, игнорируя обрезанные символы в конце
    return truncated_encoded.decode(encoding, errors='ignore')

# Применяем к столбцу 'text'
df_unique['text_truncated'] = df_unique['text'].apply(truncate_to_512_bytes)

In [136]:
tokenized = df_unique['text_truncated'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [137]:
tokenized.head(5)

0    [101, 2899, 1006, 26665, 1007, 1011, 1996, 213...
1    [101, 2899, 1006, 26665, 1007, 1011, 16824, 21...
2    [101, 2899, 1006, 26665, 1007, 1011, 1996, 256...
3    [101, 2899, 1006, 26665, 1007, 1011, 8398, 304...
4    [101, 5862, 1013, 2899, 1006, 26665, 1007, 101...
Name: text_truncated, dtype: object

### Паддинг

**Так как подаем данные в BERT батчами, поэтому все группы должны быть одинаковой длины**

In [138]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [139]:
# Делаем все группы одной длины
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [140]:
padded

array([[  101,  2899,  1006, ...,     0,     0,     0],
       [  101,  2899,  1006, ...,     0,     0,     0],
       [  101,  2899,  1006, ...,     0,     0,     0],
       ...,
       [  101,  4754, 21863, ...,     0,     0,     0],
       [  101,  7398,  2301, ...,     0,     0,     0],
       [  101,  7398,  2301, ...,     0,     0,     0]])

In [141]:
np.array(padded).shape

(44689, 236)

### Masking

**Нужно показать, что нули - это пустые добавочные символы**

In [142]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(44689, 236)

### Применение модели BERT

**Необходимые векторы находятся в переменной last_hidden_states**

In [145]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

* **padded** — это список списков токенов одинаковой длины, полученных после процедуры паддинга

* **torch.tensor(padded)** — создает из этого списка тензор, который можно подавать на вход нейросетевой модели в PyTorch

In [176]:
input_ids

tensor([[  101,  1194, 16856,  ...,     0,     0,     0],
        [  101,  1194, 16856,  ...,     0,     0,     0]])

In [177]:
attention_mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])